# Example 1 - OpenWeatherMap API — Real Sensor-like Weather Data

Provides near real-time data for temperature, humidity, wind speed, air pressure, etc.

Perfect for demonstrating regression or drift adaptation tasks (e.g., predicting next-hour temperature, energy consumption, etc.).

Free tier with public API key.

In [32]:
import requests, time
from river import linear_model, optim, preprocessing, metrics

model = preprocessing.StandardScaler() | linear_model.LinearRegression(
    optimizer=optim.SGD(0.01)
)
mae = metrics.MAE()

URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

try:
    while True:
        data = requests.get(URL).json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        x = {"wind": wind}
        y = temp

        y_pred = model.predict_one(x)
        model.learn_one(x, y)
        mae.update(y, y_pred)

        print(f"Temp={y:.2f}°C | Wind={wind:.1f} km/h | Pred={y_pred:.2f}°C | MAE={mae.get():.3f}")
        time.sleep(2)  # 5 min delay
except KeyboardInterrupt:
    print("\nStopped streaming.")


Temp=16.50°C | Wind=9.4 km/h | Pred=0.00°C | MAE=16.500
Temp=16.50°C | Wind=9.4 km/h | Pred=0.33°C | MAE=16.335
Temp=16.50°C | Wind=9.4 km/h | Pred=0.65°C | MAE=16.172
Temp=16.50°C | Wind=9.4 km/h | Pred=0.97°C | MAE=16.012
Temp=16.50°C | Wind=9.4 km/h | Pred=1.28°C | MAE=15.853
Temp=16.50°C | Wind=9.4 km/h | Pred=1.59°C | MAE=15.697
Temp=16.50°C | Wind=9.4 km/h | Pred=1.88°C | MAE=15.542
Temp=16.50°C | Wind=9.4 km/h | Pred=2.18°C | MAE=15.390
Temp=16.50°C | Wind=9.4 km/h | Pred=2.46°C | MAE=15.240
Temp=16.50°C | Wind=9.4 km/h | Pred=2.74°C | MAE=15.091
Temp=16.50°C | Wind=9.4 km/h | Pred=3.02°C | MAE=14.945
Temp=16.50°C | Wind=9.4 km/h | Pred=3.29°C | MAE=14.801
Temp=16.50°C | Wind=9.4 km/h | Pred=3.55°C | MAE=14.658
Temp=16.50°C | Wind=9.4 km/h | Pred=3.81°C | MAE=14.518
Temp=16.50°C | Wind=9.4 km/h | Pred=4.06°C | MAE=14.379
Temp=16.50°C | Wind=9.4 km/h | Pred=4.31°C | MAE=14.242
Temp=16.50°C | Wind=9.4 km/h | Pred=4.56°C | MAE=14.106
Temp=16.50°C | Wind=9.4 km/h | Pred=4.80°C | MAE

# Linear Regressuib (SGD) with ADWIN Drift Detector

In [31]:
import requests, time
from river import linear_model, optim, preprocessing, metrics, drift

# --- Model Setup ---
model = preprocessing.StandardScaler() | linear_model.LinearRegression(
    optimizer=optim.SGD(0.01)
)
mae = metrics.MAE()

# --- Drift Detector ---
adwin = drift.ADWIN(delta=0.002)

URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

print("🌦️ Starting online regression with concept drift detection. Press Ctrl+C to stop.\n")

try:
    while True:
        # Fetch current weather data
        response = requests.get(URL)
        data = response.json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        # Define input/output
        x = {"wind": wind}
        y = temp

        # Predict and learn
        y_pred = model.predict_one(x)
        model.learn_one(x, y)

        # Compute error and update detector
        error = abs(y - (y_pred or 0))
        adwin.update(error)
        mae.update(y, y_pred)

        # Check for drift (new API property name)
        drift_flag = "⚠️ Drift Detected!" if adwin.drift_detected else ""
        print(
            f"Temp={y:.2f}°C | Wind={wind:.1f} km/h | Pred={y_pred:.2f}°C "
            f"| MAE={mae.get():.3f} | Error={error:.3f} {drift_flag}"
        )

        # Reset model if drift detected
        if adwin.drift_detected:
            print("🔄 Resetting model due to detected drift...\n")
            model = preprocessing.StandardScaler() | linear_model.LinearRegression(
                optimizer=optim.SGD(0.01)
            )
            adwin = drift.ADWIN(delta=0.002)

        time.sleep(300)  # 5 min delay (API refresh)

except KeyboardInterrupt:
    print("\nStopped streaming.")


🌦️ Starting online regression with concept drift detection. Press Ctrl+C to stop.

Temp=16.50°C | Wind=9.4 km/h | Pred=0.00°C | MAE=16.500 | Error=16.500 

Stopped streaming.


# River + Evidently integration

In [24]:
# ============================================================
try:
    import evidently
except ImportError:
    !pip install evidently==0.5.1 river scikit-learn requests pandas matplotlib


In [4]:
import time, requests
import pandas as pd
from river import linear_model, optim, preprocessing, metrics, drift
from evidently.report import Report
from evidently import ColumnMapping
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from tracely import init_tracing  # for Evidently Cloud tracing

# --- Evidently Cloud connection ---
tracer_provider = init_tracing(
    address="https://app.evidently.cloud",
    api_key="dG9rbgFHWHjVjJJGn7OGiLb7In9PXF1LjoaGV4mt1uq5pfHhdQBAgH+c4XXx4RuGpJ28iYiB8p4mkAnVAmrMGhCAeoujHR3YOUD6VpFchsPvpjBKtp3nmag8FQP+l3qwGfnWTxDbG+1GnIM=",  # store in env var for security
    project_id="0199b983-1622-7224-a32f-2f608ab8d49e",
    export_name="streaming-weather-trace",
)
tracer = tracer_provider.get_tracer("streaming-weather-trace")


# --- Column mapping for Evidently ---
column_mapping = ColumnMapping()
column_mapping.target = "temperature"
column_mapping.prediction = "prediction"
column_mapping.numerical_features = ["wind"]

# --- River model setup ---
model = preprocessing.StandardScaler() | linear_model.LinearRegression(optimizer=optim.SGD(0.01))
mae = metrics.MAE()
adwin = drift.ADWIN()

# --- Data buffers for Evidently ---
reference_window = []
current_window = []

URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

try:
    while True:
        # Fetch live data
        data = requests.get(URL).json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        # Online learning
        x = {"wind": wind}
        y = temp
        y_pred = model.predict_one(x)
        model.learn_one(x, y)
        mae.update(y, y_pred)
        error = abs(y - (y_pred or 0))
        adwin.update(error)

        drift_flag = adwin.drift_detected

        # Print model status
        print(f"Temp={y:.2f}°C | Wind={wind:.1f} km/h | Pred={y_pred:.2f}°C | MAE={mae.get():.3f}")

        # Log trace
        with tracer.start_as_current_span("online_regression") as span:
            span.set_attribute("temperature", float(y))
            span.set_attribute("wind_speed", float(wind))
            span.set_attribute("prediction", float(y_pred or 0.0))
            span.set_attribute("mae", float(mae.get()))
            span.set_attribute("error", float(error))
            span.set_attribute("drift_detected", bool(drift_flag))
            span.set_attribute("timestamp", time.time())

        # Store in current buffer
        current_window.append({
            "temperature": y,
            "wind": wind,
            "prediction": y_pred,
        })

        # Once we collect enough samples, compute Evidently reports
        if len(current_window) >= 5:
            if not reference_window:
                # initialize reference window with first batch
                reference_window = current_window.copy()
                current_window = []
                print("📘 Reference window initialized with 10 samples.\n")
            else:
                # convert to DataFrames
                ref_df = pd.DataFrame(reference_window)
                cur_df = pd.DataFrame(current_window)

                # --- Regression report ---
                regression_report = Report(metrics=[RegressionPreset()], options={"render": {"raw_data": True}})
                regression_report.run(current_data=cur_df, reference_data=ref_df, column_mapping=column_mapping)
                regression_report.save_html("regression_performance.html")
                print("📊 Regression report saved.")

                # --- Data drift report ---
                drift_report = Report(metrics=[DataDriftPreset()], options={"render": {"raw_data": True}})
                drift_report.run(current_data=cur_df, reference_data=ref_df, column_mapping=column_mapping)
                drift_report.save_html("data_drift_report.html")
                print("🌡️ Drift report saved.\n")

                # Slide window forward
                reference_window = current_window.copy()
                current_window = []

        time.sleep(2)  # every minute

except KeyboardInterrupt:
    print("\nStopped streaming.")


Temp=18.00°C | Wind=2.7 km/h | Pred=0.00°C | MAE=18.000
Temp=18.00°C | Wind=2.7 km/h | Pred=0.36°C | MAE=17.820
Temp=18.00°C | Wind=2.7 km/h | Pred=0.71°C | MAE=17.642
Temp=18.00°C | Wind=2.7 km/h | Pred=1.06°C | MAE=17.467
Temp=18.00°C | Wind=2.7 km/h | Pred=1.40°C | MAE=17.294
📘 Reference window initialized with 10 samples.

Temp=18.00°C | Wind=2.7 km/h | Pred=1.73°C | MAE=17.124
Temp=18.00°C | Wind=2.7 km/h | Pred=2.05°C | MAE=16.955
Temp=18.00°C | Wind=2.7 km/h | Pred=2.37°C | MAE=16.789
Temp=18.00°C | Wind=2.7 km/h | Pred=2.69°C | MAE=16.625
Temp=18.00°C | Wind=2.7 km/h | Pred=2.99°C | MAE=16.463


/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/evidently/metrics/regression_performance/utils.py:16: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is d

📊 Regression report saved.
🌡️ Drift report saved.



/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2920: RuntimeWarning:

invalid value encountered in divide

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2920: RuntimeWarning:

invalid value encountered in divide

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7400: RuntimeWarning:

divide by zero encountered in divide

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2920: RuntimeWarning:

invalid value encountered in divide

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2920: RuntimeWarning:

invalid value encountered in divide

/home/barbon/Python/440MI/venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7400: RuntimeWarning:

divide by zero encountered in divide



AttributeError: 'Report' object has no attribute 'log_report'